In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
mol = "CO2"
mol_path = f'./{mol}'
files = os.listdir(mol_path)
l = len(files)

In [ ]:
def parse_atom_coords(file_path):
    with open(file_path, 'r') as file:
        content = file.readlines()
        file.close()
    atomic_positions = []
    start = False
    for line in content:
        if 'ATOMIC_POSITIONS' in line:
            start = True
        elif start and line.strip():  # Continue reading until a blank line is encountered
            if 'K_POINTS' in line:
                break
            atomic_positions.append([float(coord) for coord in line.split()[1:]])

    return np.array(atomic_positions)

In [ ]:
scf_in = f'{mol_path}/dis_{1}/scf.in'
def volume_calc(file_path):
    with open(file_path, 'r') as file:
        content = file.readlines()
        file.close()
    cell_units = []
    start = False
    for line in content:
        if 'CELL_PARAMETERS' in line:
            start = True
        elif start and line.strip():  # Continue reading until a blank line is encountered
            if 'ATOMIC_POSITIONS' in line:
                break
            cell_units.append([float(coord) for coord in line.split()])
    cell_units = np.array(cell_units)
    volume = round(np.linalg.det(cell_units), 7)
    return volume

In [ ]:
# coord [x, y, z] is indexed as [0, 1, 2] respectively

# Works only for SO2 and CO2
def squared_projection(p1, p2):
    squared_dist = np.sum((p1-p2)**2, axis=0)
    projs = [(p1[i] - p2[i]) ** 2 / squared_dist for i in range(3)]
    r = np.sqrt(squared_dist)
    return [projs, r]

In [ ]:
# Main

def diag_eps_reader(type, inn_path):
    df = pd.read_table(f'{inn_path}/{type}', delimiter="\s+")
    eps = df.iloc[0,1]
    return eps

def mod_polar_reader(type, inn_path):
    df = pd.read_table(f'{inn_path}/{type}', delimiter="\s+")
    mod_polar = (df.iloc[0,1] - 1) * volume
    return mod_polar

# Works only for system with two bonds
std_file_path = f'{mol_path}/{mol}_0K.txt'
if os.path.exists(std_file_path):
    stan_coord = parse_atom_coords(file_path=std_file_path)
    r1 = np.sqrt(np.sum((stan_coord[0]-stan_coord[1])**2, axis=0)) # Important to note that r1 and r2 are global variables
    r2 = np.sqrt(np.sum((stan_coord[0]-stan_coord[2])**2, axis=0)) 
    volume = 1000
    try:
        volume = volume_calc(file_path=scf_in)
    except:
        pass
else:
    raise("Error: Standard file does not exists")

def collect_data():
    equat_data = []
    for i in tqdm(range(l)):
        inn_path = f'{mol_path}/dis_{i}'
        dims = ['epsxx.dat', 'epsyy.dat', 'epszz.dat', 'scf.in']
        if os.path.exists(inn_path) and all(os.path.exists(f'{inn_path}/{dim}') for dim in dims):
            polar_data = []
            for dim in dims[:3]:
                polar = mod_polar_reader(dim, inn_path)
                polar_data.append(polar)
    
            atom_coords = parse_atom_coords(file_path=f'{inn_path}/scf.in')
            
            # We define or squared relation as "g"   -- > Multoiple bond squared function
            g1, rg1 = squared_projection(atom_coords[0], atom_coords[1]) 
            g2, rg2 = squared_projection(atom_coords[0], atom_coords[2])
    
            dr1 = rg1 - r1
            dr2 = rg2 - r2
            equat_data.append([*polar_data, *g1, *g2, dr1, dr1**2, dr2, dr2**2])
        else:
            pass
    
    clmns = ['polar_x', 'polar_y', 'polar_z',\
             'g1_x', 'g1_y', 'g1_z',\
             'g2_x', 'g2_y', 'g2_z',\
             'dr1', 'dr1_2', 'dr2', 'dr2_2']
    df_equat = pd.DataFrame(equat_data, columns=clmns)
    return df_equat

df_equat = collect_data()

In [ ]:
df_equat.head()

In [ ]:
df_equat.to_csv(f'{mol_path}/equat_{mol}.csv', index=False)

In [ ]:
# Add Abnoramlity search

def linal_sys_solve():
    coords = ['x', 'y', 'z']

    # In search of 1 global params
    all_ans = []
    total_coefs = []

    fin_params = []
    fin_exact_params = []
    for k in tqdm(range(df_equat.shape[0]//4)):
        answers = []
        all_coefs = []
        for j in range(4*k, 4*k+4):
            for i in coords:
                answers.append(df_equat[f'polar_{i}'][j])
                coefs = [(1 - df_equat[f'g1_{i}'][j]), (1 - df_equat[f'g1_{i}'][j]) * df_equat.dr1[j],\
                           (1 - df_equat[f'g1_{i}'][j]) * df_equat.dr1_2[j], (1 - df_equat[f'g2_{i}'][j]),\
                           (1 - df_equat[f'g2_{i}'][j]) * df_equat.dr2[j], (1 - df_equat[f'g2_{i}'][j]) * df_equat.dr2_2[j],\
                      df_equat[f'g1_{i}'][j], df_equat[f'g1_{i}'][j]*df_equat.dr1[j], df_equat[f'g1_{i}'][j]*df_equat.dr1_2[j],\
                      df_equat[f'g2_{i}'][j], df_equat[f'g2_{i}'][j]*df_equat.dr2[j], df_equat[f'g2_{i}'][j]*df_equat.dr2_2[j]]
                all_coefs.append(coefs)
                
                total_coefs.append(coefs)
                all_ans.append(df_equat[f'polar_{i}'][j])
                
        np_coefs = np.array(all_coefs)
        multi_params = np.linalg.lstsq(np_coefs, answers, rcond=None)[0]
        # param_list = multi_params[:(len(multi_params)//2)]
        # fin_params.append(param_list)
        fin_params.append(multi_params) # if not we don't want to repeate params

        # Exact solutions
        try:
            exact_params = np.linalg.solve(np_coefs, answers)
            fin_exact_params.append(exact_params)
        except:
            pass

    one_params = np.linalg.lstsq(total_coefs, all_ans, rcond=None)[0] # Global params
        
    return fin_params, fin_exact_params, one_params, total_coefs, all_ans

In [ ]:
fin_param, fin_exact_params, one_params, np_coefs, all_ans = linal_sys_solve()
clmns= ['a0p', 'a1p', 'a2p', 'a0l', 'a1l', 'a2l']
clmns_2 = ['a10p', 'a11p', 'a12p', 'a10l', 'a11l', 'a12l', 'a20p', 'a21p', 'a22p', 'a20l', 'a21l', 'a22l']
df_param_ap = pd.DataFrame(fin_param, columns=clmns_2)
df_param_ex = pd.DataFrame(fin_exact_params, columns=clmns_2)
print(one_params)

#### If use linalg.solve, a10p and a20p is equal as other, so we can slice the param list

#### df_param --> 6 columns ['a0p', 'a1p', 'a2p', 'a0l', 'a1l', 'a2l']

#### df_param_2 --> 12 columns ['a10p', 'a11p', 'a12p', 'a10l', 'a11l', 'a12l', 'a20p', 'a21p', 'a22p', 'a20l', 'a21l', 'a22l']

In [ ]:
mol ='SO2'
mol_path = f'./{mol}'
df_equat = pd.read_csv(f'{mol_path}/equat_{mol}.csv')

In [ ]:
pred_eps = []
dft_eps = []
pred_polar = []
for i in range(len(np_coefs)):
    a1=np.dot(np_coefs[i], one_params)
    pred_polar.append(a1)
    pred_eps.append((a1/1000) + 1)
    dft_eps.append((all_ans[i]/1000) + 1)
    
pred_polar_x = pred_polar[::3]
pred_polar_y = pred_polar[1::3]
pred_polar_z = pred_polar[2::3]


In [ ]:
pred_polar_xyz = {'pred_polar_x': pred_polar_x, 'pred_polar_y': pred_polar_y, 'pred_polar_z': pred_polar_z}
pred_polar_df = pd.DataFrame(pred_polar_xyz)
pred_polar_df.to_csv(f'{mol_path}/pred_polar_{mol}.csv', index=False)

In [ ]:
def plot(pred_eps, dft_eps):
    fig, ax = plt.subplots();
    scatter = ax.scatter(x=dft_eps, y=pred_eps);
    plt.title(f"Comparison of DFT Results with Predicted Epsilon Using Our Model for {mol}");
    plt.ylabel('Our model epsilon');
    plt.xlabel('DFT epsilon');
    plt.figure(figsize=(25,10));
    plt.show()

plot(pred_eps, dft_eps)

In [ ]:
print("MSE: ", mean_squared_error(dft_eps, pred_eps))
print("MAE: ", mean_absolute_error(dft_eps, pred_eps))

In [ ]:
df_param_ex.to_csv(f'{mol_path}/exact_param_{mol}.csv', index=False)
df_param_ap.to_csv(f'{mol_path}/approx_param_{mol}.csv', index=False)

In [ ]:
import seaborn as sns

### For CO2

In [ ]:
mol ='CO2'
mol_path = f'./{mol}'
df_param_ex = pd.read_csv(f'{mol_path}/exact_param_{mol}.csv')
df_param_ap = pd.read_csv(f'{mol_path}/approx_param_{mol}.csv')

In [ ]:
sns.heatmap(df_param_ap.corr())

In [ ]:
sns.heatmap(df_param_ex.corr())

### For SO2

In [ ]:
mol ='SO2'
mol_path = f'./{mol}'
df_param_ex = pd.read_csv(f'{mol_path}/exact_param_{mol}.csv')
df_param_ap = pd.read_csv(f'{mol_path}/approx_param_{mol}.csv')

In [ ]:
sns.heatmap(df_param_ap.corr())

In [ ]:
sns.heatmap(df_param_ex.corr())

### Number of unique sets of polarization constants for molecules

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
df_param_ex = pd.read_csv(f'{mol_path}/exact_param_{mol}.csv')
mol

In [ ]:
# Similarity of sets (Correlation) For SO2

# ex_data = df_param_ex
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_param_ex)

distortions = []
k_range = []  # You can adjust the range based on your specific dataset
k = 1
while True:
# for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init="auto")
    kmeans.fit(df_scaled)
    dist = kmeans.inertia_
    distortions.append(dist)
    k_range.append(k)
    # print(k)
    if k % 21 == 0:
        print(k)
        print(dist)
    elif dist < 1:
        print(dist)
        print(k)
        break
    k += 20


In [ ]:
# Plot the elbow
plt.plot(k_range, distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.title('Elbow Method for Optimal k')
plt.show()